In [1]:
from showupforhealth.utils import perform_train_test_split, oversample_with_smote, scale_df
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
data = pd.read_csv('/Users/alessio/code/janduplessis883/data-showup/data/output-data/full_train_data.csv')

/Users/alessio/.pyenv/versions/3.10.6/envs/showupforhealth/lib/python3.10/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [2]:
data.head(1)

,Appointment_status,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,...,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,1,17.7,0.0,40,0,0.0,0,0,0,0,...,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [3]:
X = data.drop(columns='Appointment_status')
y  = data['Appointment_status']

In [4]:
X_train, X_test, y_train, y_test = perform_train_test_split(X,y)

✅ OUTPUT: X_train, X_test, y_train, y_test
Train Set:  X_train, y_train - (721212, 36), (721212,)
 Test Set:  X_test, y_test - - (180304, 36), (180304,)


In [5]:
def standard_scaler(X_train, X_test):
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled

In [6]:
X_train_scaled, X_test_scaled = standard_scaler(X_train, X_test)

In [7]:
X_train_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)
X_train_scaled_df.head()

,temp,precipitation,Age,Sex,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,...,No_shows,Rota_ARRS,Rota_GP,Rota_HCA,Rota_Nurse,Ethnicity_Asian,Ethnicity_Black,Ethnicity_Mixed,Ethnicity_Other,Ethnicity_White
0,-0.205187,-0.267858,-0.206241,-0.760445,-0.779329,-0.503995,-0.428541,-0.279662,-0.354313,-0.639081,...,-0.065013,-0.278311,0.569313,-0.220183,-0.380814,-0.341748,-0.291692,-0.247529,-0.371143,0.752922
1,2.068924,-0.267858,1.754905,-0.760445,3.146543,1.984145,-0.428541,-0.279662,-0.354313,1.564747,...,-0.065013,-0.278311,0.569313,-0.220183,-0.380814,-0.341748,-0.291692,-0.247529,-0.371143,0.752922
2,1.253380,-0.267858,0.477880,-0.760445,0.257694,-0.503995,-0.428541,-0.279662,-0.354313,-0.639081,...,-0.467866,-0.278311,-1.756504,-0.220183,2.625956,-0.341748,-0.291692,-0.247529,-0.371143,0.752922
3,0.845609,-0.267858,1.481257,-0.760445,1.887301,-0.503995,-0.428541,-0.279662,-0.354313,-0.639081,...,-0.467866,-0.278311,0.569313,-0.220183,-0.380814,-0.341748,-0.291692,-0.247529,-0.371143,0.752922
4,0.892659,-0.267858,-0.343065,1.314994,-1.001549,-0.503995,-0.428541,-0.279662,-0.354313,-0.639081,...,-0.467866,-0.278311,0.569313,-0.220183,-0.380814,2.926129,-0.291692,-0.247529,-0.371143,-1.328159


# DL model 1

In [13]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping
model = Sequential()

es = EarlyStopping(patience=5, monitor='accuracy', restore_best_weights=True)

model.add(layers.Dense(10, activation='relu', input_dim=X.shape[1]))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(20, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

history = model.fit(X_train_scaled, y_train,
                    validation_split=0.3,
                    batch_size=32,
                    epochs=5,
                    callbacks=[es])

Epoch 1/5
15777/15777 [==============================] - 27s 2ms/step - loss: 0.1250 - accuracy: 0.9621 - val_loss: 0.1166 - val_accuracy: 0.9615
Epoch 2/5
15777/15777 [==============================] - 26s 2ms/step - loss: 0.1120 - accuracy: 0.9623 - val_loss: 0.1133 - val_accuracy: 0.9614
Epoch 3/5
15777/15777 [==============================] - 28s 2ms/step - loss: 0.1105 - accuracy: 0.9624 - val_loss: 0.1130 - val_accuracy: 0.9615
Epoch 4/5
15777/15777 [==============================] - 27s 2ms/step - loss: 0.1101 - accuracy: 0.9624 - val_loss: 0.1120 - val_accuracy: 0.9615
Epoch 5/5
15777/15777 [==============================] - 28s 2ms/step - loss: 0.1099 - accuracy: 0.9625 - val_loss: 0.1117 - val_accuracy: 0.9616


In [15]:
res = model.evaluate(X_test_scaled, y_test)
print(f'Loss: {res[0]}')
print(f'Accuracy: {res[1]}')

5635/5635 [==============================] - 5s 951us/step - loss: 0.1113 - accuracy: 0.9614
Loss: 0.11126507818698883
Accuracy: 0.9614428877830505
